In [1]:
import import_hack
import steps
from data_store import sqlite_table_datastore as sqlds
from data_store import file_system_directory_datastore as fsds
from data_store import stream_ndarray_adapter_datastore as snads
from data_store import numpy_datastore as npds
from transformer import transformers as trs
from search import exhaustive_searcher as exs
%load_ext autoreload
%autoreload 2

In [5]:
# img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
img_dir_path = r'D:\datasets\brodatz\data.brodatz\size_213x213'
sqlite_db_path='example_find_nearest_by_histograms'
sqlite_table_name='histograms'

In [6]:
image_bytes_ds = fsds.FileSystemDirectoryDataStore(dir_path=img_dir_path)
histograms_ds=sqlds.SQLiteTableDataStore(sqlite_db_path, sqlite_table_name)
image_bytes_ds.get_count()

999

In [7]:
transformers_=[trs.BytesToNdarray(), trs.NdarrayToOpencvMatrix(), trs.OpencvMatrixToHistogram(graycolor=True)]
steps.transform_step(image_bytes_ds, transformers_, histograms_ds)

In [8]:
histograms_ndarray_ds = snads.StreamNdarrayAdapterDataStore(histograms_ds)
print(histograms_ndarray_ds.get_items_sorted_by_ids().shape)
np_neighbor_ids_ds=npds.NumpyDataStore()

(999, 256, 1)


In [9]:
exs_searcher=exs.ExhaustiveSearcher(histograms_ndarray_ds.get_items_sorted_by_ids(), histograms_ndarray_ds.get_ids_sorted())
steps.search_step(histograms_ds, exs_searcher, 5 , np_neighbor_ids_ds)

In [10]:
np_neighbor_ids_ds.get_items_sorted_by_ids()

array([[  1,   3,   5,   2,   8],
       [  2,   9,   6,   3,   1],
       [  3,   2,   1,   6,   9],
       ..., 
       [997, 994, 991, 998, 995],
       [998, 994, 991, 995, 997],
       [999, 992, 996, 993, 995]])

In [13]:
ids=histograms_ndarray_ds.get_ids_sorted()
source_ids=image_bytes_ds.get_ids_sorted()
np_neighbor_source_ids=npds.NumpyDataStore()
steps.transform_step(np_neighbor_ids_ds, [trs.TranslateByKeysTransformer(ids, source_ids)], np_neighbor_source_ids)

In [14]:
np_neighbor_source_ids.get_items_sorted_by_ids()

array([['D001_01.png', 'D001_03.png', 'D001_05.png', 'D001_02.png',
        'D001_08.png'],
       ['D001_02.png', 'D001_09.png', 'D001_06.png', 'D001_03.png',
        'D001_01.png'],
       ['D001_03.png', 'D001_02.png', 'D001_01.png', 'D001_06.png',
        'D001_09.png'],
       ..., 
       ['D112_07.png', 'D112_04.png', 'D112_01.png', 'D112_08.png',
        'D112_05.png'],
       ['D112_08.png', 'D112_04.png', 'D112_01.png', 'D112_05.png',
        'D112_07.png'],
       ['D112_09.png', 'D112_02.png', 'D112_06.png', 'D112_03.png',
        'D112_05.png']], dtype=object)

In [4]:
import os
os.remove(sqlite_db_path)